In [9]:
import json
import urllib.request
import time
import datetime
today = datetime.date.today()
import tweepy as tp
from config import consumer_key,consumer_secret,access_token,access_secret

In [10]:
gameid = 2021020180
gamedetail = []
starturl = 'https://statsapi.web.nhl.com/api/v1/game/'
endurl = '/feed/live.json'

auth = tp.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tp.API(auth)

In [11]:
# time.sleep(32400)
pullurl = starturl + str(gameid) + endurl
pullurl

'https://statsapi.web.nhl.com/api/v1/game/2021020180/feed/live.json'

In [12]:
while gameid <= 2021020217:
    pullurl = starturl + str(gameid) + endurl
    with urllib.request.urlopen(pullurl) as url:
        obj = json.loads(url.read().decode())
    allPlays = obj["liveData"]["plays"]['allPlays']
    teamStats = obj["liveData"]["boxscore"]["teams"]
    game_date = obj["gameData"]["datetime"]["dateTime"]
    home_team = obj["gameData"]["teams"]["home"]["name"]
    away_team = obj["gameData"]["teams"]["away"]["name"]
    game_date = game_date.split("T")[0]
    tomorrow = today + datetime.timedelta(days = 1)
    print(f'{gameid}-{obj["gameData"]["status"]["abstractGameState"]}-{game_date}-{tomorrow} -- {home_team} vs {away_team}')
    # print(obj["gameData"]["status"]["abstractGameState"])
    x=1
    period = 1
    while obj["gameData"]["status"]["abstractGameState"] != "Final" and (home_team == 'Los Angeles Kings' or away_team == 'Los Angeles Kings') and str(game_date) == str(tomorrow):
        #print(x)
        #x=x+1
        pullurl = starturl + str(gameid) + endurl
        try:
            with urllib.request.urlopen(pullurl) as url:
                obj = json.loads(url.read().decode())
        except:
            print(f'The URL is broke, bruh. Try again.')
        allPlays = obj["liveData"]["plays"]['allPlays']
        if period < 3:
            time.sleep(5)
        hometeamscore = 0
        awayteamscore = 0
        for play in allPlays:
            period = play['about']['period']
            event = play['result']['event']
            timeee = play['about']['periodTime']
            description = play['result']['description']
            if event == 'Goal':
                if str(home_team) == 'Los Angeles Kings':
                    if play['team']['name'] == 'Los Angeles Kings':
                        hometeamscore += 1
                    else:
                        awayteamscore += 1
                if str(home_team) != 'Los Angeles Kings':
                    if play['team']['name'] != 'Los Angeles Kings':
                        hometeamscore += 1
                    else:
                        awayteamscore += 1
            if f'Period: {period}-{timeee}, {event}' not in gamedetail:
                gamedetail.append(f'Period: {period}-{timeee}, {event}')
                if event == 'Period Ready' or event == 'Hit' or event == 'Giveaway' or event == 'Takeaway' or event == 'Game Scheduled':
                    print(f'NOT TWEETED---Period: {period}-{timeee}, {event}-{description}')
                    #Period Ready
                    #Giveaway
                    #Takeaway
                    #Hit
                #elif event == 'Game Scheduled':
                    #homescore = teamStats['home']['teamStats']['teamSkaterStats']['goals']
                    #awayscore = teamStats['away']['teamStats']['teamSkaterStats']['goals']
                    #print(f'Scheduled Game: {home_team} at {away_team} #GoKingsGo')
                    #api.update_status(f'Scheduled Game: {home_team} versus {away_team} #GoKingsGo')
                    #time.sleep(8)
                elif event == 'Period Start':
                    try:
                        api.update_status(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        print(f'Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET --- Period Starting: {period} --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    time.sleep(8)
                    #Period Start
                elif event == 'Faceoff' or event == 'Blocked Shot' or event == 'Stoppage' or event == 'Period Official' or event == 'Game End' or event == 'Game Official' or event == 'Shot' or event == 'Penalty' or event == 'Missed Shot' :
                    try:
                        api.update_status(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                        print(f'Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET -- Period: {period}-{timeee}, {event}-{description} #GoKingsGo')
                    time.sleep(8)
                    #Faceoff
                    #Blocked Shot
                    #Shot
                    #Penalty
                    #Missed Shot
                    #Stoppage
                    #Period Official
                    #Game End
                    #Game Official
                elif event == 'Goal':
                    try:
                        api.update_status(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                        print(f'Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET --- Period: {period}-{timeee}, Score: {home_team}-{hometeamscore} ~ {away_team}-{awayteamscore}, {event}-{description}, #GoKingsGo')
                    time.sleep(8)
                    #Goal
                elif event == 'Period End' and (period == 1 or period == 2):
                    try:
                        api.update_status(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        print(f'Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET --- Period {period} End --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    time.sleep(1080)
                    #Period End
                elif event == 'Period End' and (period == 3):
                    try:
                        api.update_status(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        print(f'End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET --- End of Regulation --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    time.sleep(8)
                elif event == 'Period End' and (period == 4):
                    try:
                        api.update_status(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                        print(f'End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    except:
                        print(f'FAILED TO TWEET --- End of Overtime --- Score: {home_team}-{hometeamscore}, {away_team}-{awayteamscore} #GoKingsGo')
                    time.sleep(8)
                else:
                    time.sleep(0)
        print(f'{home_team}-{hometeamscore}, {away_team}-{awayteamscore}')
    gameid += 1

2021020180-Final-2021-11-09-2021-11-09 -- New York Rangers vs Florida Panthers
2021020181-Final-2021-11-09-2021-11-09 -- Washington Capitals vs Buffalo Sabres
2021020182-Preview-2021-11-10-2021-11-09 -- Boston Bruins vs Ottawa Senators
2021020183-Preview-2021-11-10-2021-11-09 -- Montréal Canadiens vs Los Angeles Kings
2021020184-Preview-2021-11-10-2021-11-09 -- Detroit Red Wings vs Edmonton Oilers
2021020185-Preview-2021-11-10-2021-11-09 -- Tampa Bay Lightning vs Carolina Hurricanes
2021020186-Preview-2021-11-10-2021-11-09 -- New Jersey Devils vs Florida Panthers
2021020187-Preview-2021-11-10-2021-11-09 -- Chicago Blackhawks vs Pittsburgh Penguins
2021020188-Preview-2021-11-10-2021-11-09 -- Winnipeg Jets vs St. Louis Blues
2021020189-Preview-2021-11-10-2021-11-09 -- Calgary Flames vs San Jose Sharks
2021020190-Preview-2021-11-10-2021-11-09 -- Vancouver Canucks vs Anaheim Ducks
2021020191-Preview-2021-11-10-2021-11-09 -- Vegas Golden Knights vs Seattle Kraken
2021020192-Preview-2021-11-

In [13]:
#api.update_status(f'Test')
for play in allPlays:
    if f'Period: {period}-{timeee}, {event}' not in gamedetail:
        gamedetail.append(f'Period: {period}-{timeee}, {event}')
        print(play)

In [6]:
# gameid = 2020020481
# gamedetail = []
# pullurl = starturl + str(gameid) + endurl
# with urllib.request.urlopen(pullurl) as url:
#     obj = json.loads(url.read().decode())
# allPlays = obj["liveData"]["plays"]['allPlays']
# for play in allPlays:
#     event = play['result']['event']
#     if event not in gamedetail:
#         print(event)
#         gamedetail.append(event)

In [7]:
# currenttt = time.localtime(time.time() + 24*3600)
# currenttt

In [ ]:
import random as rnd
for i in range(100):
    print(f'{rnd.gauss(100,15)}')
    

In [ ]:
for i in range(100):
    print(f'{rnd.randint(1,100000)}')
    

In [ ]:
 cafMiq-sabmu4-qekwyw